In [2]:
import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import accuracy_score, f1_score
from sklearn.utils import resample

# === Step 1: Load Original Dataset ===
df = pd.read_csv("anomaly_data.csv")  # 🔁 Replace with your dataset

# === Step 2: Balance the Dataset via Upsampling ===
df_majority = df[df['label'] == 1]  # anomaly
df_minority = df[df['label'] == 0]  # normal

df_minority_upsampled = resample(df_minority,
                                 replace=True,
                                 n_samples=len(df_majority),
                                 random_state=42)

df_balanced = pd.concat([df_majority, df_minority_upsampled])
df_balanced = df_balanced.sample(frac=1).reset_index(drop=True)  # Shuffle

# === Step 3: Feature Scaling ===
X = df_balanced[['temperature', 'humidity']].values
y = df_balanced['label'].values

scaler = MinMaxScaler()
X_scaled = scaler.fit_transform(X)

In [5]:
# === Step 4: Split the Data ===
X_temp, X_test, y_temp, y_test = train_test_split(X_scaled, y, test_size=0.15, random_state=42)
X_train, X_val, y_train, y_val = train_test_split(X_temp, y_temp, test_size=0.1765, random_state=42)
# → This gives approx: 70% train, 15% val, 15% test

In [7]:
# === Step 5: Build Model ===
model = tf.keras.Sequential([
    tf.keras.layers.Input(shape=(2,)),
    tf.keras.layers.Dense(8, activation='relu'),
    tf.keras.layers.Dense(4, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# === Step 6: Train Model ===
model.fit(X_train, y_train,
          epochs=30,
          batch_size=8,
          validation_data=(X_val, y_val))

Epoch 1/30
88/88 [==============================] - 1s 2ms/step - loss: 0.6998 - accuracy: 0.5293 - val_loss: 0.6907 - val_accuracy: 0.7020
Epoch 2/30
88/88 [==============================] - 0s 780us/step - loss: 0.6845 - accuracy: 0.4964 - val_loss: 0.6791 - val_accuracy: 0.4503
Epoch 3/30
88/88 [==============================] - 0s 837us/step - loss: 0.6723 - accuracy: 0.5608 - val_loss: 0.6716 - val_accuracy: 0.4503
Epoch 4/30
88/88 [==============================] - 0s 1ms/step - loss: 0.6634 - accuracy: 0.5179 - val_loss: 0.6649 - val_accuracy: 0.4503
Epoch 5/30
88/88 [==============================] - 0s 814us/step - loss: 0.6544 - accuracy: 0.5207 - val_loss: 0.6567 - val_accuracy: 0.4503
Epoch 6/30
88/88 [==============================] - 0s 818us/step - loss: 0.6458 - accuracy: 0.5851 - val_loss: 0.6472 - val_accuracy: 0.4503
Epoch 7/30
88/88 [==============================] - 0s 759us/step - loss: 0.6342 - accuracy: 0.5966 - val_loss: 0.6354 - val_accuracy: 0.5828
Epoch 8/30

In [8]:
# === Step 7: Evaluate Model ===
y_pred_probs = model.predict(X_test)
y_pred = (y_pred_probs > 0.5).astype(int)

acc = accuracy_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)

print(f"✅ Test Accuracy: {acc:.2f}")
print(f"🎯 F1 Score: {f1:.2f}")

5/5 [==============================] - 0s 676us/step
✅ Test Accuracy: 0.97
🎯 F1 Score: 0.97


In [9]:
# === Step 4: Evaluate ===
val_loss, val_acc = model.evaluate(X_val, y_val)
test_loss, test_acc = model.evaluate(X_test, y_test)
print(f"\n✅ Validation Accuracy: {val_acc*100:.2f}%")
print(f"✅ Test Accuracy: {test_acc*100:.2f}%")

5/5 [==============================] - 0s 881us/step - loss: 0.1939 - accuracy: 0.9733

✅ Validation Accuracy: 98.68%
✅ Test Accuracy: 97.33%


In [10]:
# Save the model
model.save("copy.h5")


/Users/oscarpatrikminj/Documents/IITR/Working/dep/venv/lib/python3.10/site-packages/keras/src/engine/training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [1]:
from tensorflow.keras.models import load_model
model=load_model("copy.h5")
# Load the model

# Predict on new data

In [ ]:
### INFERENCE

In [1]:
import numpy as np
import tensorflow as tf

# === Load the trained .h5 model ===
model = tf.keras.models.load_model("copy.h5")

# === Use the scaler min and max from training ===
temp_min = 5.13
temp_max = 44.99
hum_min  = 5.2
hum_max  = 99.74

# === Take manual input ===
temperature = float(input("🌡️ Enter temperature (°C): "))
humidity = float(input("💧 Enter humidity (%): "))

# === Normalize using same MinMaxScaler logic ===
temp_scaled = (temperature - temp_min) / (temp_max - temp_min)
hum_scaled = (humidity - hum_min) / (hum_max - hum_min)

# Ensure input is in correct shape
input_data = np.array([[temp_scaled, hum_scaled]])

# === Predict ===
output = model.predict(input_data)

# === Display Result ===
prob = output[0][0]
print(f"\n🔍 Model Output (Anomaly Probability): {prob:.4f}")
if prob > 0.5:
    print("⚠️ Anomaly Detected")
else:
    print("✅ Normal Condition")


1/1 [==============================] - 0s 108ms/step

🔍 Model Output (Anomaly Probability): 0.5843
⚠️ Anomaly Detected


In [18]:
import tensorflow as tf

# Load the model
model = tf.keras.models.load_model("copy.h5")

# Convert to TFLite
converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()

# Save the model
with open("copy.tflite", "wb") as f:
    f.write(tflite_model)


INFO:tensorflow:Assets written to: /var/folders/bs/x0lj933d1hv0py0d4w2ypdp40000gn/T/tmpexwl3in3/assets


INFO:tensorflow:Assets written to: /var/folders/bs/x0lj933d1hv0py0d4w2ypdp40000gn/T/tmpexwl3in3/assets
2025-06-13 20:34:06.129876: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:364] Ignored output_format.
2025-06-13 20:34:06.130082: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:367] Ignored drop_control_dependency.
2025-06-13 20:34:06.130999: I tensorflow/cc/saved_model/reader.cc:45] Reading SavedModel from: /var/folders/bs/x0lj933d1hv0py0d4w2ypdp40000gn/T/tmpexwl3in3
2025-06-13 20:34:06.131752: I tensorflow/cc/saved_model/reader.cc:91] Reading meta graph with tags { serve }
2025-06-13 20:34:06.131759: I tensorflow/cc/saved_model/reader.cc:132] Reading SavedModel debug info (if present) from: /var/folders/bs/x0lj933d1hv0py0d4w2ypdp40000gn/T/tmpexwl3in3
2025-06-13 20:34:06.134074: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:375] MLIR V1 optimization pass is not enabled
2025-06-13 20:34:06.135212: I tensorflow/cc/saved_model/load

In [19]:
converter.optimizations = [tf.lite.Optimize.DEFAULT]
quant_tflite_model = converter.convert()

with open("quant_copy.tflite", "wb") as f:
    f.write(quant_tflite_model)


INFO:tensorflow:Assets written to: /var/folders/bs/x0lj933d1hv0py0d4w2ypdp40000gn/T/tmpm0l0syl6/assets


INFO:tensorflow:Assets written to: /var/folders/bs/x0lj933d1hv0py0d4w2ypdp40000gn/T/tmpm0l0syl6/assets
2025-06-13 20:34:43.458368: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:364] Ignored output_format.
2025-06-13 20:34:43.458384: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:367] Ignored drop_control_dependency.
2025-06-13 20:34:43.458584: I tensorflow/cc/saved_model/reader.cc:45] Reading SavedModel from: /var/folders/bs/x0lj933d1hv0py0d4w2ypdp40000gn/T/tmpm0l0syl6
2025-06-13 20:34:43.459342: I tensorflow/cc/saved_model/reader.cc:91] Reading meta graph with tags { serve }
2025-06-13 20:34:43.459350: I tensorflow/cc/saved_model/reader.cc:132] Reading SavedModel debug info (if present) from: /var/folders/bs/x0lj933d1hv0py0d4w2ypdp40000gn/T/tmpm0l0syl6
2025-06-13 20:34:43.462027: I tensorflow/cc/saved_model/loader.cc:231] Restoring SavedModel bundle.
2025-06-13 20:34:43.502726: I tensorflow/cc/saved_model/loader.cc:215] Running initialization

In [22]:
import os
import time
import numpy as np
import tensorflow as tf
from tensorflow.lite.python.interpreter import Interpreter

# Paths
h5_path = "copy.h5"
tflite_path = "copy.tflite"
quant_path = "quant_copy.tflite"

# Get and print model sizes
def print_model_sizes():
    h5_size_kb = os.path.getsize(h5_path) / 1024
    tflite_size_kb = os.path.getsize(tflite_path) / 1024
    quant_size_kb = os.path.getsize(quant_path) / 1024

    print(f"\n📦 H5 Model Size: {h5_size_kb:.2f} KB")
    print(f"📦 TFLite Model Size: {tflite_size_kb:.2f} KB")
    print(f"📦 Quant Model Size: {quant_size_kb:.2f} KB")

    if quant_size_kb < 32:
        print("✅ Quantized model is suitable for Arduino Uno (size < 32KB)")
    else:
        print("❌ Quantized model is NOT suitable for Arduino Uno (size >= 32KB)")
    return quant_path

# Load interpreter & estimate runtime and memory
def evaluate_model(tflite_model_path):
    interpreter = Interpreter(model_path=tflite_model_path)
    interpreter.allocate_tensors()

    input_details = interpreter.get_input_details()
    output_details = interpreter.get_output_details()

    input_shape = input_details[0]['shape']

    # Create dummy input for test
    input_data = np.random.rand(*input_shape).astype(np.float32)

    # Warm-up
    interpreter.set_tensor(input_details[0]['index'], input_data)
    interpreter.invoke()

    # Run inference multiple times
    times = []
    for _ in range(100):
        start = time.time()
        interpreter.set_tensor(input_details[0]['index'], input_data)
        interpreter.invoke()
        end = time.time()
        times.append(end - start)

    avg_time_ms = np.mean(times) * 1000
    # Estimate memory for output tensor only
    output_tensor_index = output_details[0]['index']
    output_tensor = interpreter.tensor(output_tensor_index)()
    max_mem_kb = output_tensor.nbytes / 1024

    print(f"\n🧠 Estimated Peak Memory Use (RAM): ~{max_mem_kb:.2f} KB (for output tensor only)")
    print(f"⚡ Avg Inference Time (on this system): {avg_time_ms:.2f} ms")

# Run everything
quant_path_used = print_model_sizes()
evaluate_model(quant_path_used)



📦 H5 Model Size: 32.45 KB
📦 TFLite Model Size: 2.26 KB
📦 Quant Model Size: 2.26 KB
✅ Quantized model is suitable for Arduino Uno (size < 32KB)

🧠 Estimated Peak Memory Use (RAM): ~0.00 KB (for output tensor only)
⚡ Avg Inference Time (on this system): 0.01 ms


In [ ]:
!xxd -i quant_copy.tflite > model_data.cc
